In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**MEGA-SENA**

A Mega-Sena é uma loteria que premia aquele jogador que acerta 6, 5 ou 4 números sorteados dentre os 60 disponíveis. Os prêmios normalmente são milionários e os sorteios ocorrem duas vezes por semana. O valor mínimo de cada aposto é R$ 3,50.

O objetivo deste trabalho é fazer uma análise exploratória dos dados fornecidos pela Caixa Econômica Federal, responsável pela respectiva loteria. Nesta base, temos dados referentes ao primeiro sorteio realizado, em 11/03/1996, até o realizado no dia 25/07/2018. Dados como a Arrecadação Total, quantidade de ganhadores da Mega-Sena, Quina e Quadra, o valor do rateio, se o prêmio foi acumulado ou não, dentre outros.

**Dataframe**

Inicialmente, vou criar o dataframe e chamá-lo de Mega.

In [ ]:
Mega = pd.read_csv('/kaggle/input/sorteiosmegasena/sorteios.csv')

Mega.head(5)

Para dar continuidade, preciso saber as informações e o tamanho do arquivo 'sorteios.csv':

In [ ]:
print('Mega-Sena:', Mega.shape)

In [ ]:
Mega.info()

Como podemos observar, o arquivo CSV utilizado possui 2278 linhas e 23 colunas. 

Além disso, percebe-se que não são todos os sorteios que possuem a informação sobre a Cidade e a UF. Dos aproximados 2278 sorteios, apenas 392 contém a informação referente à Cidade e 678 sorteios possuem a UF.

Por fim, verifica-se que a coluna 'Unnamed: 22' está em branco e não possui nenhuma informação (NaN). Assim, vou excluí-la do meu banco de dados por não ser relevante para o presente estudo. 

**Limpeza da base de dados**

In [ ]:
Mega = Mega.drop('Unnamed: 22',axis=1)
print(Mega.head(5))

In [ ]:
Mega.head(5)

Em uma rápida análise das cinco primeiras linhas da base de dados, observei que os jogos do Concurso nº 3 aparecem duplicados, por exemplo.

Assim, vou excluir também a coluna referente ao ID para facilitar a identificação dos jogos que estão em duplicidade para depois apagá-los.

In [ ]:
Mega = Mega.drop('Id',axis=1)
Mega.head(5)

In [ ]:
Mega[Mega.duplicated()]

Diante disso, vou limpar a minha base e excluir todos os jogos duplicados.

In [ ]:
Mega.drop_duplicates(inplace=True)

Mega.info()

Com a exclusão dos jogos duplicados, a minha base passou a conter 2241 linhas com as informações relevantes sobre os sorteios da Mega-Sena.

Agora, vou separar a data dos sorteios realizados por 'Dia', 'Mês' e 'Ano', facilitando, assim, análises que considerarão os anos. Como a data está no formato brasileiro, precisarei transformá-la para o formato *datetime*. 

In [ ]:
Mega['data_convertida'] = Mega.iloc[:,1]
Mega.data_convertida = pd.to_datetime(Mega.data_convertida)

Mega.head(5)

In [ ]:
Mega['Dia']   = Mega.data_convertida.dt.day
Mega['Mês'] = Mega.data_convertida.dt.month 
Mega['Ano']  = Mega.data_convertida.dt.year

Mega.head(5)



Para fins de organização, colocarei o 'Ano' como sendo o meu índice.

In [ ]:
Mega.set_index('Ano')

Finalizando os ajustes dos dados do dataframe **Mega**, transformarei alguns dados que estão em formato 'object' para 'int64'. São eles: Arrecadacao_Total, Rateio_Sena, Rateio_Quina, Rateio_Quadra, Valor_Acumulado.

In [ ]:
Formatar=['Arrecadacao_Total','Rateio_Sena','Rateio_Quina','Rateio_Quadra','Valor_Acumulado']

for col in Formatar:
    Mega[col] = Mega[col].str.replace(".", "",regex=True).str.replace(",", ".")
    
Mega[Formatar] = Mega[Formatar].fillna(0).astype(float)
assert Mega[Formatar].dtypes.all() == np.float64
Mega.dtypes

 **Análise dos dados**

**Ganhadores da Mega-Sena X Mês do Ano**

Para iniciarmos, vou verificar a descrição dos Ganhadores da Mega-Sena em cada mês do ano, levando em consideração, a quantidade, a média, os valores mínimos e máximos, dentre outros.

In [ ]:
Mega.groupby('Mês')['Ganhadores_Sena'].describe()

Assim, os meses que tiveram mais ganhadores da Mega-Sena foram Janeiro (1) e Dezembro (12), sendo 15 ganhadores e 17 ganhadores, respectivamente.
Estes resultados levam a concluir que o maior número de ganhadores no mês de Dezembro se deve a um maior número de apostas, possivelmente por conta da Mega-Sena da Virada, que ocorre no dia 31/12, com o maior valor a ser pago em prêmio.

**Concurso com maior quantidade de ganhadores**

Próximo passo é verificar qual foi o jogo da Mega-Sena que teve a maior quantidade de ganhadores.

In [ ]:
Mega.nlargest(1, 'Ganhadores_Sena')

Como podemos observar, no Concurso nº 2000, ocorrido em 31/12/2017, foram 17 ganhadores da Mega-Sena. As 6 dezenas sorteadas foram: 6, 37, 34, 10, 3 e 17. O total do valor arrecadado foi de R$ 890.947.368,50.

Além disso, foram 173.428 ganhadores da Quadra que receberam R$ 423,12 cada. 

**Os 10 Concursos com o menor valor arrecadado**

In [ ]:
Mega.nsmallest(11, 'Arrecadacao_Total')

**Análises Gráficas**

Agora passarei a comparar os dados e visualizá-los em gráficos.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

**Média de Ganhadores X Ano**

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(Mega['Ano'], Mega['Ganhadores_Sena'])
plt.title('Média de Ganhadores')
plt.xticks(rotation=90)
plt.locator_params(axis='y', nbins=20)
plt.show()

Notadamente, aparecem dois outliers no gráfico acima: a média de 15 ganhadores no ano de 2004 e de 17 ganhadores no ano de 2017.
Além deles, os anos que tiveram uma maior média de ganhadores na Mega-Sena foram 2000, 2010, 2015 e 2016. Por outro lado, os anos com as menores médias de ganhadores foram os 1996, 1997, 1998, 2002, 2005, 2006 e 2007.

**Rateio da Mega-Sena X Ano**

In [ ]:
plt.figure(figsize=(15,5))
sns.pointplot(x='Valor_Acumulado', y='Ano', data=Mega, color='green')
plt.title('Rateio da Mega-Sena por ano')
plt.grid(True, color='grey')

Através deste gráfico, concluímos que o valor acumulado dos prêmios da Mega-Sena foi aumentando ao longo dos anos.

**Arrecadação Total X Ano X Mês**

In [ ]:
sns.pairplot(x_vars='Ano', y_vars='Arrecadacao_Total', data=Mega, size=10, hue='Mês')

Através deste gráfico, observa-se que o mês em que há uma maior Arrecadação é Dezembro, na maioria dos casos. 
Um dado curioso e visível é a predominância de valores zerados de arrecadação no período de 1995 até meados de 2010.

**Ganhadores da Mega-Sena X UF**

In [ ]:
sns.stripplot(x='UF', y='Ganhadores_Sena', data=Mega)
plt.xticks(rotation=90)

Diante deste gráfico, verificamos que a quantidade de ganhadores da Mega-Sena por UF é bem homogênea. 
Dois Estados se destacam por terem menos ganhadores, o TO e o RR. 
E os que possuem maiores quantidades de ganhadores da Mega-Sena (acima de 7 ganhadores) são os Estados do PR, RN, SP, RJ, MG, BA, DF, CE, PE, PB, PI e PA.

**Dezenas que mais foram sorteadas**

Por fim, verificarei quais as dezenas que mais apareceram nos sorteios da Mega-Sena.

In [ ]:
Dezenas = pd.DataFrame(Mega['1ª Dezena'].tolist() + Mega['2ª Dezena'].tolist() + Mega['3ª Dezena'].tolist() + Mega['4ª Dezena'].tolist() + Mega['5ª Dezena'].tolist() + Mega['6ª Dezena'].tolist(), columns=['numeros'])
Dezenas['numeros'].value_counts().sort_values(ascending=False).head(10).plot(kind='barh', title='Dezenas mais sorteadas', figsize=(10,5), fontsize=12, legend=True, color='gray')

**Conclusão**

Por todo o exposto, concluo que visivelmente as chances de se ganhar na Mega-Sena aumentam potencialmente no mês de Dezembro, além de serem distribuídos os maiores prêmios, principalmente por ter o maior valor arrecadado. Acredito que seja por conta da Mega-Sena da Virada, que distribui os maiores prêmios do ano. 

Além disso, fiz um levantamento das dezenas que mais aparecem nos sorteios da Mega-Sena para auxiliar nas escolhas de potencial dezena sorteada. 